In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import numpy as np
import random

# Lectura de datos
data = np.loadtxt('/content/drive/MyDrive/MÁSTER/IC/P2/data/tai256c.dat', skiprows=1)
flow = np.int32(data[:256])
distances = np.int32(data[256:])

seed = 2024
random.seed(seed)
np.random.seed(seed)

In [3]:
# Generación de la población inicial
population = np.array([np.random.permutation(np.arange(256)) for i in range(500)])

def fitness_pop(population, flow, distances):
    return np.sum(flow[np.newaxis, :, :] * distances[population[:,:,np.newaxis], population[:, np.newaxis, :]],
                  axis=(1,2))

In [4]:
# Función de selección por torneo
def tournament_selection(population, fitness_vals, tournament_size=5):
    selected = []
    for _ in range(len(population)):
        tournament = random.sample(range(len(population)), tournament_size)
        tournament_fitness = [fitness_vals[i] for i in tournament]
        winner = tournament[tournament_fitness.index(min(tournament_fitness))]
        selected.append(population[winner])
    return np.array(selected)

In [5]:
# Operador de cruce por orden (OX)
def ox_crossover(parent1, parent2):
    size = len(parent1)
    start, end = sorted(np.random.choice(range(size), 2, replace=False))

    child = -np.ones(size, dtype=int)
    child[start:end+1] = parent1[start:end+1]

    idx = 0
    for i in range(size):
        if child[i] == -1:
            while parent2[idx] in child:
                idx += 1
            child[i] = parent2[idx]
    return child

In [6]:
# Mutación por intercambio (Swap)
def swap_mutation(individual, mutation_prob=0.05):
    if random.random() < mutation_prob:
        i, j = np.random.choice(range(len(individual)), 2, replace=False)
        individual[i], individual[j] = individual[j], individual[i]
    return individual

In [7]:
def hill_climbing(individual, flow, distances, max_iterations=100):
    current_solution = individual.copy()
    current_cost = fitness_pop(np.array([current_solution]), flow, distances)[0]

    for _ in range(max_iterations):
        i, j = np.random.choice(range(len(current_solution)), 2, replace=False)
        neighbor = current_solution.copy()
        neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
        neighbor_cost = fitness_pop(np.array([neighbor]), flow, distances)[0]

        if neighbor_cost < current_cost:
            current_solution = neighbor
            current_cost = neighbor_cost
    return current_solution, current_cost

In [8]:
# Algoritmo genético con variante baldwiniana
def baldwinian_genetic_algorithm(flow, distances, population_size=1000, generations=50, crossover_prob=0.9, mutation_prob=0.05, tournament_size=5):
    population = np.array([np.random.permutation(np.arange(256)) for _ in range(population_size)])

    for generation in range(generations):
        # Evaluación baldwiniana (con hill climbing)
        fitness_vals = []
        for individual in population:
            _, cost = hill_climbing(individual, flow, distances)
            fitness_vals.append(cost)
        fitness_vals = np.array(fitness_vals)

        # Selección
        selected_population = tournament_selection(population, fitness_vals, tournament_size)

        # Cruzamiento
        offspring = []
        for i in range(0, population_size, 2):
            if random.random() < crossover_prob:
                child1 = ox_crossover(selected_population[i], selected_population[i+1])
                child2 = ox_crossover(selected_population[i+1], selected_population[i])
            else:
                child1, child2 = selected_population[i], selected_population[i+1]
            offspring.append(child1)
            offspring.append(child2)

        # Mutación
        population = np.array([swap_mutation(individual, mutation_prob) for individual in offspring])

        # Evaluación (para impresión)
        fitness_vals_print = fitness_pop(population, flow, distances)

        # Imprimir el mejor individuo de cada generación
        best_idx = np.argmin(fitness_vals_print)
        print(f"Generación {generation+1}: Mejor coste (con hill climbing) = {fitness_vals[best_idx]} , Mejor coste (sin hill climbing) = {fitness_vals_print[best_idx]}")

    # Resultados finales
    best_idx = np.argmin(fitness_vals)
    return population[best_idx], fitness_vals[best_idx]

In [9]:
# Ejecución del algoritmo
best_solution_baldwin, best_cost_baldwin = baldwinian_genetic_algorithm(flow, distances)

print("\nMejor solución encontrada (Baldwiniano) (permutación):", best_solution_baldwin)
print("Coste asociado a la mejor solución (Baldwiniano):", best_cost_baldwin)

Generación 1: Mejor coste (con hill climbing) = 49340714 , Mejor coste (sin hill climbing) = 50433218
Generación 2: Mejor coste (con hill climbing) = 50342722 , Mejor coste (sin hill climbing) = 50302038
Generación 3: Mejor coste (con hill climbing) = 49297970 , Mejor coste (sin hill climbing) = 49992488
Generación 4: Mejor coste (con hill climbing) = 50762460 , Mejor coste (sin hill climbing) = 49992488
Generación 5: Mejor coste (con hill climbing) = 50400336 , Mejor coste (sin hill climbing) = 49649336
Generación 6: Mejor coste (con hill climbing) = 51128910 , Mejor coste (sin hill climbing) = 49485318
Generación 7: Mejor coste (con hill climbing) = 49895412 , Mejor coste (sin hill climbing) = 49507972
Generación 8: Mejor coste (con hill climbing) = 50530192 , Mejor coste (sin hill climbing) = 49420448
Generación 9: Mejor coste (con hill climbing) = 49436778 , Mejor coste (sin hill climbing) = 49265224
Generación 10: Mejor coste (con hill climbing) = 49078954 , Mejor coste (sin hill 